In [ ]:
# This notebook includes VGG16 to classify if a food image is vegetarian or not vegetarian 

In [1]:
# Imports
import os
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.optimizers import adam_v2
import urllib.request
import ssl

In [2]:
# Set the path to the dataset 
train_dir = '/Volumes/SSD/dishes'
valid_dir = '/Volumes/SSD/dishes-val'
test_dir = '/Volumes/SSD/dishes-test'

In [3]:
# Define the input image size
img_width, img_height = 224, 224

# Set the number of epochs and batch size
epochs = 20
batch_size = 32

# Create a data generator for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Create a data generator for the validation set
valid_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
# Set the number of training and validation samples
nb_train_samples = 1985
nb_valid_samples = 833

In [12]:
# Download the VGG16 weights manually
weights_path = 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
if not os.path.exists(weights_path):
    url = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
    urllib.request.urlretrieve(url, weights_path)

In [13]:
# Build the VGG16 network
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [14]:
# Add a custom top layer
x = base_model.output
x = Flatten()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [15]:
# Freeze the VGG16 layers
for layer in base_model.layers:
    layer.trainable = False

In [16]:
# Compile the model
model.compile(optimizer=adam_v2.Adam(learning_rate=0.0001, decay=0.0001/epochs), loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
# Create the data generators
train_datagen = ImageDataGenerator(rescale=1.0/255)
valid_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='binary')

Found 1985 images belonging to 2 classes.
Found 833 images belonging to 2 classes.


In [18]:
# Train the model
model.fit_generator(train_generator, epochs=epochs, steps_per_epoch=nb_train_samples//batch_size, validation_data=valid_generator, validation_steps=nb_valid_samples//batch_size)

/var/folders/7_/43049s_d7v14zv0f3_1ysgqc0000gn/T/ipykernel_69765/498919934.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=epochs, steps_per_epoch=nb_train_samples//batch_size, validation_data=valid_generator, validation_steps=nb_valid_samples//batch_size)


Epoch 1/20
62/62 [==============================] - 760s 12s/step - loss: 0.6833 - accuracy: 0.5627 - val_loss: 0.6908 - val_accuracy: 0.5541
Epoch 2/20
62/62 [==============================] - 739s 12s/step - loss: 0.5983 - accuracy: 0.6866 - val_loss: 0.6887 - val_accuracy: 0.5733
Epoch 3/20
62/62 [==============================] - 744s 12s/step - loss: 0.5532 - accuracy: 0.7235 - val_loss: 0.6712 - val_accuracy: 0.5589
Epoch 4/20
62/62 [==============================] - 744s 12s/step - loss: 0.5298 - accuracy: 0.7501 - val_loss: 0.6686 - val_accuracy: 0.5853
Epoch 5/20
62/62 [==============================] - 739s 12s/step - loss: 0.4877 - accuracy: 0.8054 - val_loss: 0.6719 - val_accuracy: 0.6034
Epoch 6/20
62/62 [==============================] - 741s 12s/step - loss: 0.4682 - accuracy: 0.8111 - val_loss: 0.6635 - val_accuracy: 0.5889
Epoch 7/20
62/62 [==============================] - 731s 12s/step - loss: 0.4449 - accuracy: 0.8280 - val_loss: 0.6633 - val_accuracy: 0.5950
Epoch 

In [19]:
# Evaluate the model on the test set
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(img_width, img_height), batch_size=batch_size, class_mode='binary')
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print('Test accuracy:', test_acc)

Found 2321 images belonging to 2 classes.
73/73 [==============================] - 718s 10s/step - loss: 0.6178 - accuracy: 0.6777
Test accuracy: 0.6777251362800598
